<div>
    <img style="float:right;" src="images/snext-logo.png"/>
    <div style="float:left;color:#626262;padding-top:30px"><h1>Exercise: Simple Data Exploration in Python</h1></div>
</div>

This notebooks contains the skeleton of a simple data exploration documentation. 

Use to familiarize yourself with the process, the commands by executing the given cells. Then complete the included assignments to practices your data visualization and interpretation skills.

## 1. Choose and load dataset from database

First, we have to load data onto our digital "workbench". We use Python and the Pandas library. In case you are not familiar, please first work through the tutorials also included in this course, as this is a prerequisite to this exercise.

### Case
The CRM team of a telecommunications company is developing a customer retention campaign for private customers. To make them stay loyal to the company, free months and discounts will be offered for signing up for a contract extension.

For the campaign to be successful economically, it is necessary to only target customers with a high risk of cancelling. Otherwise, the campaign would be giving out unnecessary discounts, resulting in immediate lost revenue.

### Dataset description
The data set includes the following features, some features for the months september (sep_...) and october (oct_...):
- Aggregate of Total Revenue: The overall monthly revenue earned in Rupees by the carrier in the months August & September.
- Aggregate of SMS Revenue: The revenue earned through the SMS service used by the subscriber.
- Aggregate of Data Revenue: The revenue earned through the Data service used by the subscriber.
- Aggregate of Off Net Revenue: The revenue earned by the calls etc. made to the off-network (not the same network as the subscriber) customers by the carrier’s -present subscriber.
- Aggregate of On Net Revenue: The revenue earned by the calls etc. made to the on-network (on the same network as the subscriber) customers by the carrier’s present subscriber.-
- Network Age: The time passed since the subscriber started using the services of the carrier.
- User Type: This detail helps in knowing if the user is subscribed to a 2G (voice, no data plan) or 3G/4G/5G... ("3G") service.
- Aggregate of Complaint Count: The number of complaints made by the subscribers.
- Favorite Other Network: This information can certainly have a huge impact on churn ratio as it gives the information about which other network or operator the subscribers makes the most of the calls to and thus might influence the customer to move to that network to save money.
- Aggregate of Data Volume: The volume of the data service used by the subscriber.
- Class: Did the customer churn or remain active.

Load required libraries and jupyter extenions

In [ ]:
import pandas as pd
import requests
import matplotlib.pyplot as plt

# load Jupyter plugins to enable SQL to query data and display plots inline (below the code cell)

%load_ext sql
%matplotlib inline

In [ ]:
# The data for this exercise is contained in a sqlite database that is compressed with ZIP
# ZIP file is expected to be in folder data 

# Uncompress database / skip this if you downloaded and unzipped the database manually
import zipfile
zipfile.ZipFile('data/snext-exercises.zip', 'r').extractall('data')

In [ ]:
%sql sqlite:///data/snext-exercises.db

Load anonymized records from an asian telco provider to practice data exploration.

In [ ]:
data = %sql SELECT * FROM telco_churn_asia
df = data.DataFrame()

... and check the dataframe content ...

In [ ]:
df.head()

## 2. Explore dataset with descriptive statistics


In [ ]:
# Generate descriptive statistics for all numeric features
df.describe()

In [ ]:
# Look at a nominal features
df.aug_user_type.value_counts()

In [ ]:
# Analyze monthly revenue
df_active_customers = df.copy()
df_active_customers = df_active_customers[df_active_customers.network_age >= 0]  # we have customers with negative network_age, who have booked a service but are not using it yet
df_active_customers["monthly_rev"] = df_active_customers.Aggregate_Total_Rev / (df_active_customers.network_age/30)
df_active_customers.monthly_rev.describe()

In [ ]:
# Divide age (# of days the customer uses the service) by 30 to get months and describe the resulting dataset
(df.network_age/30).describe()

In [ ]:
# Look at outliers of total revenue
print("10% of customers with less than ", df.Aggregate_Total_Rev.quantile(0.1), " total revenue")
print("10% of customers with more than ", df.Aggregate_Total_Rev.quantile(0.9), " total revenue")

---
### <span style="color:#46B7E9;">Assignment: Analyze further customer attributes to answer key questions</span>
1. Think about how to answer these further questions
   - What are favorite other networks and did the distribution change in the observed timeframe?
   - How much data volume did the customers use and are there outliers that we should know about when designing data plans?
2. Develop 2 further questions that can be answered with the dataset.
3. Think about the scale of measurement (nominal, ordinal, metric) of the variable you are analyzing and find answers to your questions using the dataframe object.

If feeling unsure, ask AI chatbots or google code examples to find commands or patterns you can't remember from the introduction.

Now check: Do you have a rough understanding about what kinds of customers are in the dataset?

## 3. Visualize single variables

In [ ]:
df.aug_user_type.value_counts().plot(kind="bar", title="Service Subscription in August", xlabel="Kind of service", ylabel="# of customers")

In [ ]:
df.Aggregate_Total_Rev.plot(kind="box", title="Distribution of Total Revenue", ylabel="USD")

In [ ]:
# we have to remove some extreme outliers for the boxplot to be better readable
df[df.Aggregate_Total_Rev < 5000].Aggregate_Total_Rev.plot(kind="box", title="Distribution of Total Revenue", ylabel="USD")

---
### <span style="color:#46B7E9;">Assignment: Visualize the answers to the questions you developed</span>
To dig a little deeper and present the information in a better digestable way, now try to visualize the answers to the questions you developed in chapter 2.

1. Think about the scale (nominal, ordinal, metric), the question you have in mind and which chart type would be a good fit.
2. If unsure, get inspiration from the [Python Graph Gallery](https://www.python-graph-gallery.com). Keep in mind, we started using matplotlib as plotting library, some examples are for seaborn.
3. Bring the diagram into shape, add formatting, labels etc. When unsure about commands, check the documentation of pandas ```plot()``` command [here](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.html) or google for "matplotlib cheat sheet" or ask your preferred AI chatbot.

## 4. Analyze and visualize multiple variables

In [ ]:
# do customers switch service or is this stable in the observed timeframe?
pd.crosstab(df.aug_user_type, df.sep_user_type, margins=True)

In [ ]:
# what are favored networks of churned vs. active customers? Callers to which networks did churn more often?
pd.crosstab(df.Class, df.aug_fav_a, margins=True)

In [ ]:
# do customers with many calls also use more data? do we see a correlation?
df.loc[:,["Aggregate_Calls", "Aggregate_Data_Vol"]].corr()

In [ ]:
# Important predictor for churn is the ratio of onnet revenue and offnet revenue. Customers who primarily call people in other networks are probably more likely to churn.
# Let's analyze all non-outlier customers (remove those exceeding the 90% quantile)
# Are there customers in the upper left corner of the plot (way more offnet revenue compared to onnet revenue)

df["avg_daily_onnet_rev"] = df.Aggregate_ONNET_REV / df.network_age
df["avg_daily_offnet_rev"] = df.Aggregate_OFFNET_REV / df.network_age

df[df.network_age > 0].plot(kind="scatter", title="Off-net vs On-net total revenue", 
           x="avg_daily_onnet_rev", y="avg_daily_offnet_rev",
            xlim=[0,100], ylim=[0,100],
            xlabel="Average Daily On-Net Revenue", ylabel="Average Daily Off-Net Revenue")

In [ ]:
(df.Aggregate_Calls/df.network_age).plot(kind="hist", bins=100,  xlim=[0,5])

---
### <span style="color:#46B7E9;">Assignment: Try to find more interesting relationships between the included features</span>
1. Output the ```df.head()``` again and look at the variables. Think about what relationships and you expect to be in the data? What could be relevant to explain churn?
2. Write down guiding questions in a markdown cell.
3. Output the statistics or diagrams to answer your questions.